In [ ]:
# ---
# title: 09. Hierarchical Risk Parity (HRP)
# tags: [Optimization, HRP, Clustering, Finance]
# difficulty: Advanced
# ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import scipy.cluster.hierarchy as sch
from scipy.spatial.distance import squareform

# Import our optimizer
import sys
sys.path.append('..')
from src.analytics.advanced_optimizer import AdvancedOptimizer
from src.analytics.portfolio_optimizer import PortfolioOptimizer

sns.set_style('darkgrid')
plt.rcParams['figure.figsize'] = (12, 6)

# Hierarchical Risk Parity (HRP)

## Introduction

**Hierarchical Risk Parity (HRP)** is an advanced portfolio construction technique developed by Marcos López de Prado that addresses key limitations of Mean-Variance Optimization (MVO):

### Problems with Traditional MVO:
1. **Instability**: Small changes in expected returns lead to large changes in optimal weights
2. **Concentration**: Tends to produce concentrated portfolios (few assets dominate)
3. **Estimation Error**: Highly sensitive to estimation errors in the covariance matrix

### HRP Solution:
HRP uses **machine learning clustering** to build diversified portfolios based on the hierarchical structure of asset correlations, without requiring expected return estimates.

## The Three-Step HRP Algorithm

1. **Tree Clustering**: Use hierarchical clustering on the correlation matrix to identify asset groups
2. **Quasi-Diagonalization**: Reorganize the covariance matrix so similar assets are adjacent
3. **Recursive Bisection**: Allocate capital recursively using inverse-variance weighting

## Step 1: Load Market Returns Data

In [ ]:
# Load returns from silver layer
silver_path = Path("../data/silver")
files = list(silver_path.glob("market_returns_*.parquet"))

if not files:
    raise FileNotFoundError("No returns data found. Run data ingestion first.")

latest = max(files, key=lambda f: f.stat().st_mtime)
returns = pd.read_parquet(latest)

print(f"Loaded returns for {len(returns.columns)} tickers")
print(f"Date range: {returns.index[0]} to {returns.index[-1]}")
print(f"Shape: {returns.shape}")

# Use a subset for clearer visualization
# Select top 20 tickers by average volume or use all
returns_subset = returns.iloc[:, :20]  # First 20 tickers
print(f"\nUsing {len(returns_subset.columns)} tickers for demonstration")

## Step 2: Visualize Correlation Structure

Before applying HRP, let's examine the correlation structure of our assets.

In [ ]:
# Calculate correlation matrix
corr = returns_subset.corr()

# Plot correlation heatmap
plt.figure(figsize=(14, 10))
sns.heatmap(corr, cmap='RdYlGn', center=0, 
            square=True, linewidths=0.5,
            cbar_kws={"shrink": 0.8})
plt.title('Asset Correlation Matrix (Before Clustering)', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print(f"Average correlation: {corr.values[np.triu_indices_from(corr.values, k=1)].mean():.3f}")

## Step 3: Hierarchical Clustering

HRP uses the **distance metric**: $d_{i,j} = \sqrt{2(1 - \rho_{i,j})}$

Where $\rho_{i,j}$ is the correlation between assets $i$ and $j$. This transforms correlation into a proper distance metric.

In [ ]:
# Convert correlation to distance
dist = np.sqrt(2 * (1 - corr))
dist_condensed = squareform(dist, checks=False)

# Perform hierarchical clustering (single linkage)
linkage = sch.linkage(dist_condensed, method='single')

# Plot dendrogram
plt.figure(figsize=(16, 8))
dendro = sch.dendrogram(linkage, labels=returns_subset.columns.tolist(),
                        leaf_font_size=10, leaf_rotation=90)
plt.title('Hierarchical Clustering Dendrogram', fontsize=14, fontweight='bold')
plt.xlabel('Assets', fontsize=12)
plt.ylabel('Distance', fontsize=12)
plt.axhline(y=1.0, color='r', linestyle='--', alpha=0.5, label='Example Cut Height')
plt.legend()
plt.tight_layout()
plt.show()

print("\n📊 Dendrogram Interpretation:")
print("- Assets that merge at lower heights are more similar (highly correlated)")
print("- The tree structure reveals natural groupings in the portfolio")
print("- HRP uses this hierarchy to allocate weights recursively")

## Step 4: Compute HRP Weights

Now we apply the full HRP algorithm using our `AdvancedOptimizer` class.

In [ ]:
# Initialize optimizer
optimizer = AdvancedOptimizer()

# Compute HRP weights
hrp_weights = optimizer.get_hrp_weights(returns_subset)

print("\n=== HRP Portfolio Weights ===")
print(hrp_weights.sort_values(ascending=False))
print(f"\nSum of weights: {hrp_weights.sum():.6f}")
print(f"Number of non-zero positions: {(hrp_weights > 0.001).sum()}")

## Step 5: Compare HRP vs Mean-Variance Optimization

Let's compare HRP weights with traditional MVO (Max Sharpe) weights.

In [ ]:
# Compute MVO weights for comparison
mvo_optimizer = PortfolioOptimizer()
mvo_weights = mvo_optimizer.get_max_sharpe_weights(returns_subset)

# Create comparison dataframe
comparison = pd.DataFrame({
    'HRP': hrp_weights,
    'MVO (Max Sharpe)': mvo_weights
}).fillna(0)

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# HRP weights
comparison['HRP'].sort_values(ascending=True).plot(kind='barh', ax=axes[0], color='steelblue')
axes[0].set_title('HRP Weights', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Weight', fontsize=12)
axes[0].grid(axis='x', alpha=0.3)

# MVO weights
comparison['MVO (Max Sharpe)'].sort_values(ascending=True).plot(kind='barh', ax=axes[1], color='coral')
axes[1].set_title('MVO (Max Sharpe) Weights', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Weight', fontsize=12)
axes[1].grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.show()

# Calculate concentration metrics
def herfindahl_index(weights):
    """Calculate Herfindahl-Hirschman Index (concentration measure)"""
    return (weights ** 2).sum()

hrp_hhi = herfindahl_index(hrp_weights)
mvo_hhi = herfindahl_index(mvo_weights)

print("\n=== Portfolio Concentration ===")
print(f"HRP Herfindahl Index: {hrp_hhi:.4f}")
print(f"MVO Herfindahl Index: {mvo_hhi:.4f}")
print(f"\nLower HHI = More diversified")
print(f"HRP is {((mvo_hhi - hrp_hhi) / mvo_hhi * 100):.1f}% more diversified than MVO")

## Step 6: Visualize Quasi-Diagonalized Correlation Matrix

After HRP reorders assets by similarity, the correlation matrix becomes more block-diagonal.

In [ ]:
# Get the sorted order from HRP
# We'll recreate the quasi-diag ordering
link = sch.linkage(dist_condensed, 'single')
sorted_ix = optimizer.get_quasi_diag(link)
sorted_tickers = returns_subset.columns[sorted_ix].tolist()

# Reorder correlation matrix
corr_sorted = corr.loc[sorted_tickers, sorted_tickers]

# Plot before and after
fig, axes = plt.subplots(1, 2, figsize=(18, 7))

# Original
sns.heatmap(corr, cmap='RdYlGn', center=0, ax=axes[0],
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
axes[0].set_title('Original Correlation Matrix', fontsize=14, fontweight='bold')

# Quasi-diagonalized
sns.heatmap(corr_sorted, cmap='RdYlGn', center=0, ax=axes[1],
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
axes[1].set_title('Quasi-Diagonalized (HRP Ordered)', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 Notice how similar assets are now grouped together (block structure)")
print("This organization enables efficient recursive bisection for weight allocation")

## Step 7: Portfolio Performance Metrics

In [ ]:
def portfolio_stats(weights, returns, name="Portfolio"):
    """Calculate portfolio statistics"""
    portfolio_returns = (returns * weights).sum(axis=1)
    
    ann_return = portfolio_returns.mean() * 252
    ann_vol = portfolio_returns.std() * np.sqrt(252)
    sharpe = ann_return / ann_vol if ann_vol > 0 else 0
    
    return {
        'Strategy': name,
        'Annual Return': ann_return,
        'Annual Volatility': ann_vol,
        'Sharpe Ratio': sharpe
    }

# Calculate stats for both portfolios
hrp_stats = portfolio_stats(hrp_weights, returns_subset, "HRP")
mvo_stats = portfolio_stats(mvo_weights, returns_subset, "MVO")

# Equal weight baseline
equal_weights = pd.Series(1/len(returns_subset.columns), index=returns_subset.columns)
equal_stats = portfolio_stats(equal_weights, returns_subset, "Equal Weight")

# Display comparison
stats_df = pd.DataFrame([hrp_stats, mvo_stats, equal_stats]).set_index('Strategy')
print("\n=== Portfolio Performance Comparison ===")
print(stats_df.to_string())

# Visualize
stats_df.plot(kind='bar', figsize=(12, 6), rot=0)
plt.title('Portfolio Performance Metrics', fontsize=14, fontweight='bold')
plt.ylabel('Value', fontsize=12)
plt.legend(loc='upper left')
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

## Key Takeaways

### ✅ Advantages of HRP:
1. **Stability**: Weights are more stable to input changes (no matrix inversion)
2. **Diversification**: Naturally produces well-diversified portfolios
3. **No Return Estimates**: Only requires covariance/correlation (more robust)
4. **Interpretability**: Tree structure provides intuitive asset groupings

### ⚠️ Limitations:
1. **No Return Optimization**: Doesn't explicitly maximize expected returns
2. **Linkage Method**: Results depend on choice of linkage (single, complete, average)
3. **Equal Risk Assumption**: Assumes equal risk contribution within clusters

### 🎯 When to Use HRP:
- When expected returns are unreliable or unavailable
- For large universes where MVO becomes unstable
- When diversification is the primary goal
- In high-frequency or systematic strategies requiring stability

### 📚 Further Reading:
- López de Prado, M. (2016). "Building Diversified Portfolios that Outperform Out of Sample"
- "Advances in Financial Machine Learning" - Chapter on HRP